<a href="https://colab.research.google.com/github/pullz6/Research_inspired_Self-Explaining-Neural-Networks-for-Business-Process-Monitoring/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [47]:
#Importing libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime
from sklearn.preprocessing import MinMaxScaler

In [48]:
df = pd.read_csv("Helpdesk.csv")

In [49]:
df.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,Assign seriousness,complete,Value 2,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,Take in charge ticket,complete,Value 2,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,Resolve ticket,complete,Value 2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
4,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21348 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   concept:name          21348 non-null  object
 1   lifecycle:transition  21348 non-null  object
 2   org:resource          21348 non-null  object
 3   time:timestamp        21348 non-null  object
 4   Activity              21348 non-null  object
 5   Resource              21348 non-null  object
 6   case:concept:name     21348 non-null  object
 7   case:variant          21348 non-null  object
 8   case:variant-index    21348 non-null  int64 
 9   case:creator          21348 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.6+ MB


In [51]:
df.drop_duplicates(inplace=True)

In [52]:
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], format='mixed')

<ipython-input-52-ac090aac2b9b>:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], format='mixed')


In [53]:
df['new_date'] = [d.date() for d in df['time:timestamp']]
df['new_time'] = [d.time() for d in df['time:timestamp']]

In [54]:
min_timestamps = df.groupby('case:concept:name')['time:timestamp'].transform('min')

In [55]:
df['time_lapsed'] = df['time:timestamp'] - min_timestamps

In [56]:
df['time_lapsed'] = pd.to_timedelta(df['time_lapsed'])

In [57]:
df['time_lapsed'] = df['time_lapsed'].dt.total_seconds()

In [58]:
df['concept:name'].unique()

array(['Assign seriousness', 'Take in charge ticket', 'Resolve ticket',
       'Closed', 'Wait', 'Create SW anomaly', 'Insert ticket',
       'Schedule intervention', 'RESOLVED', 'INVALID', 'VERIFIED',
       'Resolve SW anomaly', 'Require upgrade', 'DUPLICATE'], dtype=object)

In [59]:
#Checking if we can concept:name and activity is the same as well as the org:resource and the resource, so that we can delete this feild.
df['Checker_1'] = (df['concept:name'] == df['Activity'])
df['Checker_2'] = (df['org:resource'] == df['Resource'])

In [60]:
#Let's see if it all the same, if True.
df['Checker_2'].unique()

array([ True])

In [61]:
#Lets drop it since it is the same
df.drop(['concept:name','org:resource','Checker_1','Checker_2','case:creator','lifecycle:transition','case:variant'],axis=1, inplace=True)

In [43]:
#Lets create a stage number for each activity stage of the help desk, this will help us easily identify where the case is in terms of the process

#This is also to immulate the label encoder
#First create a dataframe with stage numbers and the stage names
stage = pd.DataFrame()
stage['Stage_no'] = [1,2,3,4,5,6,7,8,9,9.1,9.2,10]
stage['Activity'] = ['Insert ticket','Assign seriousness','Take in charge ticket','Wait','Require upgrade','Schedule intervention','Resolve ticket','VERIFIED','RESOLVED','INVALID', 'DUPLICATE','Closed']

In [62]:
df=pd.merge(df,stage,on='Activity', how='left')

In [63]:
df.head()

,time:timestamp,Activity,Resource,case:concept:name,case:variant-index,new_date,new_time,time_lapsed,Stage_no
0,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,33,2010-01-13,08:40:25,0.0,2.0
1,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,33,2010-01-29,08:52:27,1383122.0,3.0
2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,33,2010-01-29,08:52:34,1383129.0,7.0
3,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,33,2010-02-13,08:52:48,2679143.0,10.0
4,2010-01-13 12:26:04+02:00,Assign seriousness,Value 2,Case2748,1,2010-01-13,12:26:04,0.0,2.0


In [64]:
df.describe()

,case:variant-index,time_lapsed,Stage_no
count,21221.000000,2.122100e+04,21142.000000
mean,14.502427,1.167490e+06,5.274544
std,35.928597,1.483251e+06,3.076202
min,1.000000,0.000000e+00,1.000000
25%,1.000000,2.400000e+01,3.000000
50%,2.000000,2.707180e+05,4.000000
75%,7.000000,2.519608e+06,7.000000
max,226.000000,5.183565e+06,10.000000


One-hot-Encoding and Preprocessing

In [65]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [66]:
encoder = OneHotEncoder(sparse_output=False)

In [67]:
activities = df[['Activity']]
one_hot_encoded = encoder.fit_transform(activities)

In [68]:
encoded_df = pd.DataFrame(
    one_hot_encoded,
    columns=encoder.get_feature_names_out(['Activity'])
)

In [69]:
df = pd.concat([df.drop('Activity', axis=1), encoded_df], axis=1)

In [70]:
df.head()

,time:timestamp,Resource,case:concept:name,case:variant-index,new_date,new_time,time_lapsed,Stage_no,Activity_Assign seriousness,Activity_Closed,...,Activity_INVALID,Activity_Insert ticket,Activity_RESOLVED,Activity_Require upgrade,Activity_Resolve SW anomaly,Activity_Resolve ticket,Activity_Schedule intervention,Activity_Take in charge ticket,Activity_VERIFIED,Activity_Wait
0,2010-01-13 08:40:25+02:00,Value 2,Case3608,33,2010-01-13,08:40:25,0.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-01-29 08:52:27+02:00,Value 2,Case3608,33,2010-01-29,08:52:27,1383122.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2010-01-29 08:52:34+02:00,Value 2,Case3608,33,2010-01-29,08:52:34,1383129.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,2010-02-13 08:52:48+02:00,Value 5,Case3608,33,2010-02-13,08:52:48,2679143.0,10.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-01-13 12:26:04+02:00,Value 2,Case2748,1,2010-01-13,12:26:04,0.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
df.columns

Index(['time:timestamp', 'Resource', 'case:concept:name', 'case:variant-index',
       'new_date', 'new_time', 'time_lapsed', 'Stage_no',
       'Activity_Assign seriousness', 'Activity_Closed',
       'Activity_Create SW anomaly', 'Activity_DUPLICATE', 'Activity_INVALID',
       'Activity_Insert ticket', 'Activity_RESOLVED',
       'Activity_Require upgrade', 'Activity_Resolve SW anomaly',
       'Activity_Resolve ticket', 'Activity_Schedule intervention',
       'Activity_Take in charge ticket', 'Activity_VERIFIED', 'Activity_Wait'],
      dtype='object')

In [72]:
X_sequences = []
y_act_sequences = []
y_time_sequences = []

In [74]:
case_groups = df.groupby('case:concept:name')

for name, group in case_groups:
    group = group.sort_values('time:timestamp')

before

In [146]:
case_groups = df.groupby('case:concept:name')
max_len = 15  # As per your model's expected timesteps
num_features = 26  # Must match your feature count

In [148]:
df['new_date'] = pd.to_datetime(df['new_date'])

df['new_date'].fillna(method='ffill', inplace=True)
df['new_time'].fillna(method='ffill', inplace=True)
df['time_lapsed'] = df['time_lapsed'].astype(dtype='Int64')

for _, group in case_groups:
    group = group.sort_values('time:timestamp')

    # Create feature matrix for each case
    features = np.column_stack([
        group[activities].values,
        group['time_lapsed'].values.reshape(-1, 1),
        group['new_date'].dt.dayofweek.values / 6,
        group['new_date'].dt.hour.values / 23
    ])

    # Pad/truncate to max_len
    padded = pad_sequences([features], maxlen=max_len, padding='post', truncating='post', dtype='float32')[0]
    X_sequences.append(padded)

    # Get targets (modify as needed)
    y_act.append(pd.get_dummies(group['Stage_no']).values[-1])  # Last activity
    y_time.append(group['time_lapsed'].values[-1])  # Last time

<ipython-input-148-80772a1594b2>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['new_date'].fillna(method='ffill', inplace=True)
<ipython-input-148-80772a1594b2>:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['new_date'].fillna(method='ffill', inplace=True)
<ipython-input-148-80772a1594b2>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behav

TypeError: float() argument must be a string or a real number, not 'NaTType'

Model building

In [305]:
!pip install keras

In [306]:
#Creating the model architecture

In [307]:
#Importing the required libriares since the LSTM model will be done with Pytorch
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.utils import get_file
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import BatchNormalization

In [308]:
temp = df.groupby('case:concept:name')
max_len = 0
for name, groups in temp:
  max_len = max(max_len, len(groups.index))
print(max_len)

15


In [309]:
num_features = len(df['Stage_no'].unique()) + 13
main_input = Input(shape=(max_len, num_features), name='main_input')

In [310]:
main_input

<KerasTensor shape=(None, 15, 26), dtype=float32, sparse=False, name=main_input>

In [311]:
input_data = df[['new_date','new_time','time_lapsed','Activity_Assign seriousness', 'Activity_Closed',
       'Activity_Create SW anomaly', 'Activity_DUPLICATE', 'Activity_INVALID',
       'Activity_Insert ticket', 'Activity_RESOLVED',
       'Activity_Require upgrade', 'Activity_Resolve SW anomaly',
       'Activity_Resolve ticket', 'Activity_Schedule intervention',
       'Activity_Take in charge ticket', 'Activity_VERIFIED', 'Activity_Wait']]

In [312]:
#Create the training split

In [313]:
l1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, dropout=0.2)(main_input) # the shared layer
b1 = BatchNormalization()(l1)
l2_1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)
l2_2 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in time prediction
b2_2 = BatchNormalization()(l2_2)
act_output = Dense(len(df['Stage_no'].unique()), activation='softmax', kernel_initializer='glorot_uniform', name='act_output')(b2_1)
time_output = Dense(1, kernel_initializer='glorot_uniform', name='time_output')(b2_2)

In [314]:
model = Model(inputs=[main_input], outputs=[act_output, time_output])

opt = Nadam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, weight_decay=0.004, clipvalue=3)

model.compile(loss={'act_output':'categorical_crossentropy', 'time_output':'mae'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=42)
model_checkpoint = ModelCheckpoint('output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [315]:
print("X_train dtype:", X_train.dtype)
print("y_a_train dtype:", y_a_train.dtype)
print("y_t_train dtype:", y_t_train.dtype)

X_train dtype: float32
y_a_train dtype: float64
y_t_train dtype: float64


In [316]:
y_t_train

array([0.26682833, 0.26682968, 0.51685336, ..., 0.23401057, 0.2789011 ,
       0.52892498])

In [317]:
model.fit(X_train, {'act_output':y_a_train, 'time_output':y_t_train}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, epochs=500)

Epoch 1/500


ValueError: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, 15, 26), found shape=(None, 17)